<h1>Start by activating our project enviornment and installing required dependencies</h1>

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate() 
##Make sure Julia can see our module 
push!(LOAD_PATH, "./src/")

###Load key functionality 
###This will take a while the first time you do it 
using RadarQC

  Activating project at `~/Documents/Grad_School/Research/RadarQC/RadarQC`
[ Info: Precompiling RadarQC [top-level]


<h2>Let's begin by splitting our CFRadials into a Training and a Testing directory</h2>
<h2><span style="color:Red">WARNING: This function will begin by DELETING the training and testing directories to clean them</span></h2>

In [11]:
###Make sure to use absolute paths here 
CASE_PATHS= ["/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/BAMEX", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/HAGUPIT", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/RITA", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/VORTEX"]

TRAINING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TRAINING"
TESTING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

"/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

In [12]:

split_training_testing!(CASE_PATHS, TRAINING_PATH, TESTING_PATH)


TOTAL NUMBER OF TDR SCANS ACROSS ALL CASES: 1780
TESTING SCANS PER CASE 89
NUMBER OF SCANS IN CASE: 482
TRAINING GROUP SIZE: 196 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 228 ASSIGNED TRAINING
 INDEXES 229 TO 257 ASSIGNED TESTING
 INDEXES 258 TO 453 ASSIGNED TRAINING
 INDEXES 454 TO 482 ASSIGNED TESTING
Total length of case files: 482
Length of testing files: 89 - 0.18464730290456433 percent
Length of testing_files: 393 - 0.8153526970954357 percent
NUMBER OF SCANS IN CASE: 238
TRAINING GROUP SIZE: 74 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 106 ASSIGNED TRAINING
 INDEXES 107 TO 135 ASSIGNED TESTING
 INDEXES 136 TO 209 ASSIGNED TRAINING
 INDEXES 210 TO 238 ASSIGNED TESTING
Total length of case files: 238
Length of testing files: 89 - 0.3739495798319328 percent
Length of testing_files: 149 - 0.6260504201680672 percent
NUMBER OF SCANS IN CASE: 794
TRAINING GROUP SIZE

<h2>Now, calculate input features based off of the training/testing data<h2> 
<h1><span style="color:Green">This takes up a lot of memory in a notebook, so I would recommend running this in a script</span></h1>




In [2]:
###Arguments are
###Path to training file/directory 
###Path to file containing arguments on features to calculate 
###Where to output calculated features 

###This will use the 5-parameter model described in the referenced manuscript 

###You can specified the verbose=false flag so as to not overload notebook memory
###Generally diagnostics about timing will print out 


calculate_features("./CFRADIALS/CASES/TRAINING", "./MODELS/DesRosiers_Bell_23/config.txt", "training_features.h5"; 
                    verbose=true, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
                    REMOVE_HIGH_PGG = true, remove_variable = "VV" )
calculate_features("./CFRADIALS/CASES/TESTING", "./MODELS/DesRosiers_Bell_23/config.txt", "testing_features.h5"; 
                    verbose=false, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
                    REMOVE_HIGH_PGG = true, remove_variable = "VV")

OUTPUTTING DATA IN HDF5 FORMAT TO FILE: training_features.h5
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222027.272_to_19950516_222030.101_TA-ELDR_AIR.nc in 2.6836609840393066 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222028.676_to_19950516_222031.494_TF-ELDR_AIR.nc in 0.16089987754821777 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222030.112_to_19950516_222032.931_TA-ELDR_AIR.nc in 0.1375260353088379 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222031.505_to_19950516_222034.335_TF-ELDR_AIR.nc in 0.14379215240478516 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222032.942_to_19950516_222035.760_TA-ELDR_AIR.nc in 0.14464807510375977 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222034.346_to_19950516_222037.164_TF-ELDR_AIR.nc in 0.14263916015625 seconds
Processed ./CFRADIALS/CASES/TRAINING/cfrad.19950516_222035.771_to_19950516_222038.601_TA-ELDR_AIR.nc in 0.14021611213684082 seconds
Processed ./CFRADIAL

In [3]:
###Remove validation 
###keyword args 
###training_output -- path to output training set to 
###validation_output -- path to output validation set to 
remove_validation("training_features.h5")

## Now - train the model! This could take on the order of 20-30 minutes if training on something the size of the ELDORA dataset

In [6]:
###Now, let's train our model on it! 
RadarQC.train_model("train_no_validation_set.h5", "trained_model.joblib";
             verify=true, verify_out="model_verification.h5")


Opening HDF5.File: (read-only) train_no_validation_set.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 1646.114226102829 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 95.0%

SAVING MODEL: 
WRITING VERIFICATION DATA TO model_verification.h5


<h1>Now, select variables you wish to QC and apply the model to them!</h1> 

In [9]:
###These need to have the same name as in the cfradial file
VARS_TO_QC = ["ZZ", "VV"]
###This will write out the new QC'ed variables to the same cfrad file with the name ZZ_QC and VV_QC 
QC_scan("/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/SMALL_TEST/cfrad.19950516_221950.411_to_19950516_221953.219_TA-ELDR_AIR.nc", 
        "./MODELS/DesRosiers_Bell_23/config.txt", 
        "trained_model.joblib"; 
        VARIABLES_TO_QC = VARS_TO_QC, 
        QC_suffix="_QC")


PROCESSING: /Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/SMALL_TEST/cfrad.19950516_221950.411_to_19950516_221953.219_TA-ELDR_AIR.nc

Completed in 0.228438138961792 seconds
Already exists... overwriting

REMOVED 6648 PRESUMED NON-METEORLOGICAL DATAPOINTS
FINAL COUNT OF DATAPOINTS IN ZZ: 6938
Already exists... overwriting

REMOVED 6660 PRESUMED NON-METEORLOGICAL DATAPOINTS
FINAL COUNT OF DATAPOINTS IN VV: 6938
